## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,38.80,87,100,6.69,overcast clouds
1,1,Upernavik,GL,72.7868,-56.1549,40.59,94,97,9.10,overcast clouds
2,2,Rikitea,PF,-23.1203,-134.9692,74.50,77,10,13.44,clear sky
3,3,Palora,EC,-1.7000,-77.9333,68.05,88,82,3.49,light rain
4,4,Yetkul,RU,54.8221,61.5880,53.58,80,2,8.95,clear sky


In [23]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = float(input("What is the maximun temperature you would like for your trip? "))
min_temp = float(input("What is the minimun temperature you would like for your trip? "))

What is the maximun temperature you would like for your trip? 100
What is the minimun temperature you would like for your trip? 50


In [24]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,74.50,77,10,13.44,clear sky
3,3,Palora,EC,-1.7000,-77.9333,68.05,88,82,3.49,light rain
4,4,Yetkul,RU,54.8221,61.5880,53.58,80,2,8.95,clear sky
5,5,Port Alfred,ZA,-33.5906,26.8910,58.87,73,60,10.38,broken clouds
9,9,Ponta Do Sol,PT,32.6667,-17.1000,74.01,87,83,5.91,broken clouds


In [25]:
# 4a. Determine if there are any empty rows.
cities_preferred_df.count()

City_ID                583
City                   583
Country                578
Lat                    583
Lng                    583
Max Temp               583
Humidity               583
Cloudiness             583
Wind Speed             583
Current Description    583
dtype: int64

In [26]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis = "index",how="any")
print(len(clean_df))
clean_df.head()

578


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,74.50,77,10,13.44,clear sky
3,3,Palora,EC,-1.7000,-77.9333,68.05,88,82,3.49,light rain
4,4,Yetkul,RU,54.8221,61.5880,53.58,80,2,8.95,clear sky
5,5,Port Alfred,ZA,-33.5906,26.8910,58.87,73,60,10.38,broken clouds
9,9,Ponta Do Sol,PT,32.6667,-17.1000,74.01,87,83,5.91,broken clouds


In [27]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,74.50,clear sky,-23.1203,-134.9692,
3,Palora,EC,68.05,light rain,-1.7000,-77.9333,
4,Yetkul,RU,53.58,clear sky,54.8221,61.5880,
5,Port Alfred,ZA,58.87,broken clouds,-33.5906,26.8910,
9,Ponta Do Sol,PT,74.01,broken clouds,32.6667,-17.1000,
11,Falmouth,GB,59.90,scattered clouds,50.1544,-5.0711,
12,Sivaki,RU,53.58,clear sky,54.8221,61.5880,
13,Severo-Kurilsk,RU,53.58,clear sky,54.8221,61.5880,
14,Ambilobe,MG,68.41,broken clouds,-13.2000,49.0500,
16,Port Elizabeth,ZA,58.87,broken clouds,-33.5906,26.8910,


In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        nan = hotel_df.loc[index, "Hotel Name"]

In [33]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis ="index", how = "any")
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,74.50,clear sky,-23.1203,-134.9692,People ThankYou
3,Palora,EC,68.05,light rain,-1.7000,-77.9333,Rancho grande
4,Yetkul,RU,53.58,clear sky,54.8221,61.5880,Баден-Баден Термы Еткуль
5,Port Alfred,ZA,58.87,broken clouds,-33.5906,26.8910,The Halyards Hotel
9,Ponta Do Sol,PT,74.01,broken clouds,32.6667,-17.1000,Hotel do Campo
...,...,...,...,...,...,...,...
712,Ostrovnoy,RU,53.58,clear sky,54.8221,61.5880,Баден-Баден Термы Еткуль
713,Tual,ID,77.81,scattered clouds,-8.5833,116.1167,Hotel Santika Mataram
714,Mehamn,NO,66.25,overcast clouds,62.5833,6.1333,Flem Cabins
716,Diamantino,BR,64.35,broken clouds,-20.3297,-40.2925,Quality Suites Vila Velha


In [34]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig